This notebook projects a 3D point cloud from lidar onto a camera image.

In [ ]:
import cv2
import numpy as np
import pandas as pd
from cv_bridge import CvBridge
import plotly.express as px
import plotly.graph_objects as go
import ros_numpy
import rosbag
from tf.transformations import quaternion_matrix
# %load_ext blackcellmagic

#### **The following values need to be adjusted for each rosbag**

How to get transformation from lidar to camera frame:
1) Add `static_transform_publisher` with zed and lidar transforms to base_link to a launch file and launch it
2) Run `get_tfs.py` which returns the translation `(x, y, z)` and rotation `(qx, qy, qz, qw)` between two frames
3) Copy them from terminal
4) Insert them below

In [ ]:
# Rosbag path
bag_path = '/home/user/rosbags/final/slam/u_c2s_half_odom_hdl.bag'
# Load rosbag
bag = rosbag.Bag(bag_path)

# ROS topics
camera_topic = '/zed2i/zed_node/left/image_rect_color'
lidar_topic = '/velodyne_points'

# Frame to show (is equivalent to time: frame/10 s)
frame = 1

# Transformation from lidar to camera left optical (using output from tf node)
trans = [0.06500000000000006, -0.029994648094052723, -0.4211903145677902]
quat = [-0.452267597655442, 0.452267597655442, -0.5435568232585948, -0.5435568232585948]

Helper functions

In [ ]:
def unpack_bag(bag_path, topic):
    """Extracts all msgs (with time) of a topic from a rosbag"""
    # Load rosbag
    bag = rosbag.Bag(bag_path)

    t_msg = []
    msgs = []
    for topic, msg, t in bag.read_messages(topics=topic):
        msgs.append(msg)
        # Time at which msg was recorded
        t_msg.append(t.to_time())
    return msgs, t_msg


def synchronize_topics(topic1, t_topic1, topic2, t_topic2, t_thresh=0.1):
    """Synchronizes two topics (even if different rate)."""
    if len(topic1) == len(topic2):
        return topic1, topic2

    # Make sure both topics start around the same time and remove first values of
    # signal which is ahead if this is not the case
    topic1, t_topic1, topic2, t_topic2 = correct_for_time_diff(
        topic1, t_topic1, topic2, t_topic2)

    # Assume first array is bigger
    array_big = t_topic1
    array_small = t_topic2
    is_topic1_bigger = True
    # Swap if not
    if len(t_topic1) < len(t_topic2):
        array_small, array_big = array_big, array_small
        is_topic1_bigger = False
    indices = []

    # For each time of the small array, check which is
    # the closest msg in the big array
    for i, v in enumerate(array_small):
        # Offset big array with a msg from small array
        diff = np.abs(array_big - v)
        # Index where time difference is the smallest
        idx = diff.argmin()
        if idx < len(array_small) and diff[idx] < t_thresh:
            indices.append(idx)

    print(
        "t_start_diff: {:.3f} s, t_end_diff: {:.3f} s".format(
            np.abs(array_small[0] - array_big[indices][0]),
            np.abs(array_big[indices][-1] - array_small[len(array_big[indices] - 1)]),
        )
    )

    if is_topic1_bigger:
        topic1_synched = topic1[indices]
        topic2_synched = topic2[: len(topic1_synched)]
        return topic1_synched, topic2_synched
    else:
        topic2_synched = topic2[indices]
        topic1_synched = topic1[: len(topic2_synched)]
        return topic1_synched, topic2_synched


def correct_for_time_diff(topic1, t_topic1, topic2, t_topic2, t_thresh=0.1):
    """Checks for a time offset of two topics at the beginning and shorts ahead signal"""
    # Assume first topic is behind and convert to numpy array
    topic_behind, topic_ahead = np.asarray(topic1), np.asarray(topic2)
    t_topic_behind, t_topic_ahead = np.asarray(t_topic1), np.asarray(t_topic2)
    is_topic1_behind = True
    # Swap if other way around
    if t_topic2[0] - t_topic1[0] > t_thresh:
        topic_ahead, topic_behind = topic_behind, topic_ahead
        t_topic_ahead, t_topic_behind = t_topic_behind, t_topic_ahead
        is_topic1_behind = False
    # No offset
    elif np.abs(t_topic1[0] - t_topic2[0]) < t_thresh:
        return tuple([np.asarray(x) for x in [topic1, t_topic1, topic2, t_topic2]])

    # Search for time of ahead topic which is closest to start of behind topic
    diff = np.abs(t_topic_ahead - t_topic_behind[0])
    # Index where time difference is the smallest
    idx = diff.argmin()
    # Remove first couple values to make both start at the same time
    topic_ahead = topic_ahead[idx:]
    t_topic_ahead = t_topic_ahead[idx:]

    if is_topic1_behind:
        print("topic1 was behind by {}".format(idx))
        return topic_behind, t_topic_behind, topic_ahead, t_topic_ahead
    else:
        print("topic2 was behind by {}".format(idx))
        return topic_ahead, t_topic_ahead, topic_behind, t_topic_behind


def cut_pc(pc2D, x1=1280, y1=720, idx=None):
    """Cuts points of 2D pointcloud which lie outside of camera image"""
    if idx is None:
        idx = (
            (pc2D[:, 0] > 0) & (pc2D[:, 0] < x1) & (pc2D[:, 1] > 0) & (pc2D[:, 1] < y1)
        )
        pc_smol = pc2D[idx]
        return pc_smol, idx
    return pc2D[idx]

Load data (camera image and pointcloud) and extract one frame

In [ ]:
# ZED camera image
image, t_image = unpack_bag(bag_path, camera_topic)
# Velodyne point cloud
lidar, t_lidar = unpack_bag(bag_path, lidar_topic)

# Synchronize both topics (zed camera has 30 fps whereas lidar is only 10 Hz)
image, lidar = synchronize_topics(image, t_image, lidar, t_lidar)

# Extract one frame
img = image[frame]
pc = lidar[frame]

# Convert from image msg to numpy array
bridge = CvBridge()
img = bridge.imgmsg_to_cv2(img, desired_encoding="passthrough")
# Convert point cloud msg to numpy array
pc_array = ros_numpy.point_cloud2.pointcloud2_to_xyz_array(pc, remove_nans=True)

How to get camera parameters:
1) Start rosbag with zed recording and echo `camera_info` topic
2) Copy important information
    1) D are the distortion parameters (**needed**)
    2) K is the intrinsic camera matrix (for the raw (distorted) images) (**needed**)
    3) R is the Rectification matrix 
    4) P is the projection/camera matrix
3) Insert them below

In [ ]:
# Lidar points
# Remove points behind car (because not visible in camera frame)
pc_array = pc_array[pc_array[:, 0] > 0]
objectPoints = pc_array

# Get rotation vector from lidar frame to camera frame
# Convert quaternion to rotation matrix
rotMat = quaternion_matrix(quat)[:3, :3]
# Convert rotation matrix to rotation vector
rvec, _ = cv2.Rodrigues(rotMat)

# Translation vector from lidar to camera frame
tvec = np.asarray(trans)

# Camera matrix (K)
cameraMatrix = np.array(
    [
        [519.9877319335938, 0.0, 630.9716796875],
        [0.0, 519.9877319335938, 352.2940673828125],
        [0.0, 0.0, 1.0],
    ]
)

# Distortion coefficients (D)
distCoeffs = np.array([0.0, 0.0, 0.0, 0.0, 0.0])

Project 3D lidar points onto 2D plane

In [ ]:
# Projection of 3D points onto 2D plane
lidar_img_points, _ = cv2.projectPoints(
    objectPoints, rvec, tvec, cameraMatrix, distCoeffs)
lidar_img_points = lidar_img_points.reshape(len(lidar_img_points), 2)

# Remove points which do not lie inside image plane
lidar_img_points_small, indices = cut_pc(lidar_img_points, img.shape[1], img.shape[0])
# Same for original pointcloud (used for distance)
pc_array_small = cut_pc(pc_array, idx=indices)

# Create dataframe with x, y coordinates and distance of lidar points
df = pd.DataFrame(lidar_img_points_small, columns=["x", "y"])
# Add distance information
df["dist"] = pc_array_small[:, 0]

Visualize

In [ ]:
fig = px.imshow(img)
# fig.add_trace(px.scatter(lidar_img_points_small[:, 0], lidar_img_points_small[:, 1]))
fig.add_trace(
    go.Scatter(
        x=df["x"],
        y=df["y"],
        mode="markers",
        marker=dict(size=3, opacity=0.8, color=df["dist"]),
    )
)
fig.update_layout(width=1920 * 0.7, height=1080 * 0.7)
fig.show()